## Process:
### 2.上传stopwords，defword，jieba分词
### 1.将类别分类，并下采样
### 3.if-idf选取特征词
### 4.构建稀疏矩阵
### 5.调用ridge regression/xgboost/randomforest等
### 6.求均值
### 7.调用Boosting优化。。

# 调包导入数据

In [1]:
import pandas as pd
import numpy as  np
import jieba

In [243]:
train_df=pd.read_csv(r'H:\比赛\景区口碑评价分值预测\train_first.csv',engine='python',encoding='utf-8')
predict_df=pd.read_csv(r'H:\比赛\景区口碑评价分值预测\predict_first.csv',engine='python',encoding='utf-8')
data = pd.concat([train_df, predict_df],ignore_index=True)

In [3]:
train_df.head()

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,5
1,f4d51947-eac4-3005-9d3c-2f32d6068a2d,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,4
2,74aa7ae4-03a4-394c-bee0-5702d3a3082a,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,4
3,099661c2-4360-3c49-a2fe-8c783764f7db,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,5
4,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼一日游,5


# 调用停用词和自定义词并切分

In [4]:
jieba.load_userdict(r'H:\比赛\newdirect.txt')
stopwords = [line.strip() for line in open('H:\比赛\stop_words_ch.txt', 'r').readlines()]  
def jieba_fenci(raw, stopwords_list):
    # 使用结巴分词把文件进行切分
    b = set()
    word_set = set(jieba.lcut(raw, cut_all=False))
    #word_list=list(set(word_list))
    for word in word_set:
        if word in stopwords_list:
            b.add(word)
    # word_set用于统计A[nClass]
    #word_list.remove('\n')
    word_list = list(word_set- b)
    return word_list

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\kate\AppData\Local\Temp\jieba.cache
Loading model cost 0.972 seconds.
Prefix dict has been built succesfully.


In [244]:
lst=[]
for wordlist in data['Discuss']:
    word_list=jieba_fenci(wordlist,stopwords)
    lst.append(word_list)
data['Discuss']=lst
data['Discuss']=data['Discuss'].apply(lambda x: ' '.join(word for word in x))

In [242]:
data

,Id,Discuss,Score
16526,53644183-1a04-3458-a087-c5f1b79eb92c,天气 雾 烟雾 不好 缭绕,4
15220,855e73d1-c554-38d4-be11-b6b413bc1010,小巷 走过 一幅 画 狭窄 吹 湖 南湖 一种 乐趣 闲逛 湖边 眼前一亮 之后 不失为 徽...,4
25259,aca89f04-ec6c-3849-bc96-cd22f631d180,普达措 香格里拉 原味 数一数二 逛一逛 效果 天气 公园 国家 绝对 单论 拍 照片 去过...,5
15791,b89fa5c8-7877-3de6-8c5c-441e41d04cc1,74 东城区 教堂 第二所 宣武门 又名 继 俗称 耶稣会 北京 兴建 八面 王府井 号 天...,4
15372,f5a73aeb-90e5-339a-8172-a7cc915fb5e2,温泉 不错,4
20127,b54588a4-a0cf-3435-967b-4141577cf693,很 值得 一去 地方 推荐 好 景色,4
14926,bab2d528-7654-3726-919b-9e76cea97926,景点 非常 推荐 九寨沟,4
32556,112865cf-2679-3271-9b97-b4bccc930b17,一周 2.8 绕湖 东西 平方千米 名胜区 文化遗产 观赏性 少数几个 风景名胜 唯一 约 ...,5
29415,11490ea4-aa71-38ff-b9f3-ddd73664962d,庄严 金碧辉煌 法相 远 海面 即可 目睹,5
11731,ed4ab7ce-e9df-3864-944f-3579568b8b6e,春节 出门,4


In [245]:
train_df=data.loc[:99999]
predict_df=data.loc[100000:]

In [246]:
train_df.to_csv(r'H:\比赛\景区口碑评价分值预测\ridge\datasave\traincut.csv')
predict_df.to_csv(r'H:\比赛\景区口碑评价分值预测\ridge\datasave\predictcut.csv')


In [241]:
predict_df

,Id,Score
100000,9a1caf96-681e-3c11-b588-43ac742d7fd2,5.000000
100001,82b450db-65c2-351c-84fb-761d76582680,4.658296
100002,2eec4606-590c-3fa2-b846-7f92441c54a6,4.147733
100003,509f9a68-ac41-35ff-9d2e-2fc12f73ed7f,4.203718
100004,395f4b22-1c5f-328a-a19d-5065e0530cbc,5.000000
100005,6f1202b2-0c1a-3d79-a675-453aad931e3e,3.765184
100006,b4bc951f-ce1e-32ae-b27c-bbecba1fd0ee,4.128602
100007,30369496-e4b1-30b9-b427-fe23fd1712d6,3.672541
100008,704a36e8-47b1-3b93-bb6a-43610c81e1a0,4.256715
100009,8a5711e2-b855-3088-adcc-a4d634e2cbb9,5.000000


In [9]:
train_df_1=train_df.loc[train_df['Score']==1,:].reset_index(drop=True)
train_df_2=train_df.loc[train_df['Score']==2,:].reset_index(drop=True)
train_df_3=train_df.loc[train_df['Score']==3,:].reset_index(drop=True)
train_df_4=train_df.loc[train_df['Score']==4,:].reset_index(drop=True)
train_df_5=train_df.loc[train_df['Score']==5,:].reset_index(drop=True)

# 3份第四类

In [10]:
train_df_4

,Id,Discuss,Score
0,f4d51947-eac4-3005-9d3c-2f32d6068a2d,成立 深刻 来说 新 意义 重要 中国 举行,4
1,74aa7ae4-03a4-394c-bee0-5702d3a3082a,好看 泉莫属 一去 瀑布 多个 非 有名 三叠 非常 推荐 最 庐山,4
2,3b7f3f2e-886f-3a68-a810-2c37cfd728d3,方便,4
3,88914409-bd13-3d47-b5a2-691177dde8fd,如画 古人 感受 走平路 来说 看水 秋景 身体 可以 智慧结晶 美丽 状况不佳 红黄绿 看...,4
4,973afeca-7530-3f56-b7f5-bef36d889025,护城河 绍兴 夜游,4
5,81502f08-b884-38b8-8169-7de7a0680a82,值得 不到 方便 里面 好长时间 周围 看一看 交通 五分之一 太大 走,4
6,988f2319-3292-305a-aaaf-ba18bc397e5a,可惜 古典 欣赏 苏州 玩玩 不错 园林 总是 一下 逛逛 感冒 不太,4
7,f6d82a8c-ef72-3a0a-95ca-95aa2fbb7f7d,长城 好歹,4
8,a4dc34f1-6a97-3b86-829c-466cdaa86bf2,崇祯帝 处 俯瞰 绝佳 自缢 登顶 故宫,4
9,20e13219-7e4e-3279-b3f7-f82d29a6e993,好看 洞 美 非常 龙王,4


In [11]:
train_df_41=train_df_4.loc[:int(len(train_df_4)/3)]
train_df_42=train_df_4.loc[int(len(train_df_4)/3):int(len(train_df_4)*2/3)]
train_df_43=train_df_4.loc[int(len(train_df_4)*2/3):int(len(train_df_4))]
train_df4=[train_df_41,train_df_42,train_df_43]

# 5份第五类

In [12]:
 train_df_5

,Id,Discuss,Score
0,201e8bf2-77a2-3a98-9fcf-4ce03914e712,已经 玩够 第四次 第三 感觉 公园 游乐 好大 没有,5
1,099661c2-4360-3c49-a2fe-8c783764f7db,波光粼粼 宁静 一起 气势磅礴 北京 最贵 雄伟 门票 地方 美 颐和园 相比 故宫 最值 觉得,5
2,97ca672d-e558-3542-ba7b-ee719bba1bab,迪斯尼 一日游,5
3,bf13ec92-6079-3451-ade3-88020cb0dcb5,赞,5
4,489c3d94-9c44-3cf2-949c-1b507c374c69,糟点 唯一,5
5,285bba78-16a3-3c1d-b648-baa483883ee3,周边游 周未,5
6,e7801d96-73d0-35c4-9e00-cc15caaa384a,一定 jrvytqlamf 排队 景点 不错 太长 做好 攻 烟火 好玩 晚上 真的 服务 ...,5
7,cd91dc2f-2331-3c73-bc8d-da027337270d,不错 一日游 感觉 选择,5
8,7ce97eca-63a8-30a1-9687-6796f34606f1,hai 有趣 xing,5
9,25e21097-bd41-3589-b12c-62bc7b04eb6d,留下来 照片 br 荡气回肠 年 必然 10 再 居然 没有,5


In [13]:
train_df_51=train_df_5.loc[:int(len(train_df_5)/5)]
train_df_52=train_df_5.loc[int(len(train_df_5)/5):int(len(train_df_5)*2/5)]
train_df_53=train_df_5.loc[int(len(train_df_5)*2/5):int(len(train_df_5)*3/5)]
train_df_54=train_df_5.loc[int(len(train_df_5)*3/5):int(len(train_df_5)*4/5)]
train_df_55=train_df_5.loc[int(len(train_df_5)*4/5):int(len(train_df_5))]
train_df5=[train_df_51,train_df_52,train_df_53,train_df_54,train_df_55]

# 合并

In [142]:
train15=[]
for bag4 in train_df4:
    trainbag=pd.concat([train_df_1,train_df_2,train_df_3,bag4],ignore_index=True)
    print(len(trainbag))
    for bag5 in train_df5:
        trainbag0=pd.concat([trainbag,bag5],ignore_index=True)
        print(len(trainbag0))
        train15.append(trainbag0)

20601
32621
32621
32622
32621
32621
20601
32621
32621
32622
32621
32621
20601
32621
32621
32622
32621
32621


In [143]:
train15[10].head()

,Id,Discuss,Score
0,52b057ed-a885-3f3e-8c42-cdc2197f8d02,2.1 学生票 共定 有变 风景区 一百多 没人接 今天 钱 申请 客服 1.31 普通 四...,1
1,80e3560c-eb9f-39b3-bc32-840af9343c7d,不值,1
2,1e727667-3961-372a-ae61-f0a7ce1874e5,景区 不 提起 最 愿意,1
3,9edee53c-e328-3367-a6b5-b82e27b862a5,换上 走过 回收 第一天 试运营 透明 统一 玻璃 订 桥面 一点 磨痕 套 已经 还好 间...,1
4,46ea7b32-0dc5-3322-8bf0-8936da458fa1,还贵 上面 东西 差 卫生 字 吃 破,1


# 打乱顺序

In [150]:
from sklearn.utils import shuffle
train15=list(map(lambda x :shuffle(x),train15))
train15=list(map(lambda x :pd.DataFrame(x),train15))

In [17]:
from sklearn.feature_extraction.text import CountVectorizer ,TfidfTransformer
#feature_extraction =TfidfVectorizer(norm='l2')
vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值  
tfidf=transformer.fit_transform(vectorizer.fit_transform(train15[10]['Discuss']))
tfidf

<32621x48101 sparse matrix of type '<class 'numpy.float64'>'
	with 396251 stored elements in Compressed Sparse Row format>

In [55]:
train10=train15[10]

,Id,Discuss,Score
0,52b057ed-a885-3f3e-8c42-cdc2197f8d02,2.1 学生票 共定 有变 风景区 一百多 没人接 今天 钱 申请 客服 1.31 普通 四...,1
1,80e3560c-eb9f-39b3-bc32-840af9343c7d,不值,1
2,1e727667-3961-372a-ae61-f0a7ce1874e5,景区 不 提起 最 愿意,1
3,9edee53c-e328-3367-a6b5-b82e27b862a5,换上 走过 回收 第一天 试运营 透明 统一 玻璃 订 桥面 一点 磨痕 套 已经 还好 间...,1
4,46ea7b32-0dc5-3322-8bf0-8936da458fa1,还贵 上面 东西 差 卫生 字 吃 破,1
5,870cd4ac-ec0d-360d-8681-1e48bd062f49,影响 心情,1
6,21a75f0d-513a-39a5-96b7-3ec82e0e9879,收费 白排 两个 太坑 开放 竟然 小时 鄙视 毁 日 一上午 光出 停车场 景区 商家 队...,1
7,3e5cfd25-ee4c-33af-bf39-b8a8fbe7b68c,全 告示 观看 一些 做法 公共服务 评论 开玩笑 今天 钟意 深刻 20 选择 购买 国家...,1
8,728eb490-a4a3-3ff2-a2a0-0224629340ea,菜市场 景区 象个 整个,1
9,fea8f6cd-f3ba-3f3a-b8b4-41420bff281b,孙中山 里面 级 之后 台阶 有着 先生 雕像 陵墓 329 没有,1


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from scipy.sparse import csr_matrix, hstack

In [68]:
from  sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

In [79]:
cv = CountVectorizer()
discuss = cv.fit_transform(train15[2]['Discuss'])
tf = TfidfVectorizer(max_df=10000),norm='l2')
discuss_tf = tf.fit_transform(train15[2]['Discuss'])
# length = csr_matrix(pd.get_dummies(data['length'],sparse=True).values)
data =hstack((discuss, discuss_tf)).tocsr()

In [80]:
X_train=data[:30000]
y_train=train15[2]['Score'][:30000]

In [86]:
ridge=Ridge(3.15)
ridge.fit(X_train,y_train)
pred=ridge.predict(data[30000:])
#test_score =1/(1+np.sqrt(-cross_val_score(ridge,X_train,y_train,cv=10,scoring='mean_squared_error')))

In [96]:
np.sqrt(pow(pred-train15[2]['Score'][30000:],2).sum()/2622)

0.6180962364616198

array([0.37041585, 0.50373826, 0.5020165 , 0.58143881, 0.62965513,
       0.63307426, 0.53649222, 0.50100642, 0.50373542, 0.50662689])

In [154]:
predictscore=[]
tf = TfidfVectorizer()
#cv = CountVectorizer()

#X_test=tf.transform(train15[0]['Discuss'])
y_test=train15[0]['Score']
for data in train15:
    X_train =tf.fit_transform(data['Discuss'])
    print('over')
    y_train=data['Score']
    X_test=tf.transform(train15[0]['Discuss'])
    ridge=Ridge(3.15)
    ridge.fit(X_train,y_train)
    pred=ridge.predict(X_test)
    score=np.sqrt(pow(pred-(y_test),2).sum()/len(data))
    predictscore.append(score)
print(predictscore)

over
over
over
over
over
over
over
over
over
over
over
over
over
over
over
[0.7324852927032823, 0.7752049861399757, 0.7748798326803641, 0.7748087390036356, 0.775283602590986, 0.7395175904708557, 0.7817173256919803, 0.7813460460909499, 0.7809403784292229, 0.78149668977078, 0.7397517409607337, 0.7819199806793965, 0.7816115735356746, 0.7810655003807837, 0.781615024609057]


In [166]:
predict=[]
tf = TfidfVectorizer()
#cv = CountVectorizer()

#X_test=tf.transform(train15[0]['Discuss'])
#y_test=train15[0]['Score']
for data in train15:
    X_train =tf.fit_transform(data['Discuss'])
    print('over')
    y_train=data['Score']
    X_test=tf.transform(predict_df['Discuss'])
    ridge=Ridge(3)
    ridge.fit(X_train,y_train)
    pred=ridge.predict(X_test)
    #score=np.sqrt(pow(pred-(y_test),2).sum()/len(data))
    predict.append(pred)
print(predict)

over
over
over
over
over
over
over
over
over
over
over
over
over
over
over
[array([4.77528318, 4.55656018, 4.27383005, ..., 3.47755036, 3.88831056,
       3.8721922 ]), array([4.72582571, 4.80253982, 4.1016788 , ..., 3.44820812, 3.81186453,
       4.02565608]), array([4.78595996, 4.59443418, 4.00518351, ..., 3.83890883, 3.88928566,
       3.93719995]), array([4.76703794, 4.644142  , 4.22539538, ..., 3.43217538, 3.99498553,
       3.78432293]), array([4.85558132, 4.51261948, 4.07111076, ..., 3.47405059, 3.67349277,
       4.03284668]), array([4.86771903, 4.58181533, 4.28955369, ..., 3.49411289, 3.86318785,
       3.77752483]), array([4.84336366, 4.79045774, 4.12915455, ..., 3.50383213, 3.87177857,
       3.9075405 ]), array([4.88715239, 4.59342349, 4.03994611, ..., 3.89917146, 3.86614041,
       3.84060459]), array([4.85175716, 4.6867616 , 4.24253648, ..., 3.48025064, 3.98865086,
       3.69842919]), array([4.94605838, 4.50789792, 4.0546802 , ..., 3.53175811, 3.6682405 ,
       3.907127

In [212]:
prediction=(np.mat(predict).sum(0)/15)

In [214]:
prediction[prediction>4.7]=5

In [163]:
alphas=2.6
test_scores=[]
for alpha in alphas:
    clf =Ridge(alpha)
    test_score =1/(1+np.sqrt(-cross_val_score(clf,X_train,y_train,cv=10,scoring='mean_squared_error')))
    test_scores.append(np.mean(test_score))
test_scores

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
i:\python\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring metho

[0.5431459600970124,
 0.5431509647459127,
 0.5431487667289454,
 0.5431414953600309,
 0.5431284375767804,
 0.5431110931118515]

In [164]:
test_scores

[0.5431459600970124,
 0.5431509647459127,
 0.5431487667289454,
 0.5431414953600309,
 0.5431284375767804,
 0.5431110931118515]

In [122]:
predictscore

[0.7771555973410724,
 0.7765668188759759,
 0.7765787566057023,
 0.7772928963459197,
 0.7385971847763452,
 0.782538131216068,
 0.7818090182481813,
 0.781580632229813,
 0.7824854116868144,
 0.738617455984869,
 0.7825148607185015,
 0.7819245299045039,
 0.781499222853359,
 0.7824314652010449]

In [232]:
ridgescore=prediction.tolist()[0]

In [248]:
#del predict_df['Discuss']
predict_df ['Score']=ridgescore

i:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [236]:
predict_df.to_csv(r'H:\比赛\景区口碑评价分值预测\ridge\pred_ridge_2018312.csv',header=None ,index=False)

In [250]:
del predict_df['Id']

In [252]:
predict_df.to_csv(r'H:\比赛\景区口碑评价分值预测\ridge\预测值.csv' ,index=False)